In [1]:

from ProcessingEmbeddings import *
from HardDebias import *
from DoubleHardDebias import *
from INLP import *
from utils import *

In [2]:
import codecs
import json
import operator

In [3]:
glove=Embeddings('glove-wiki-gigaword-50')

Loading glove-wiki-gigaword-50 embeddings
vectors shape: (400000, 50), word2idx length: 400000, vocab length: 400000


In [4]:
vectors=glove.vectors
word2idx=glove.word2idx
vocab=glove.words
dict_vectors = glove.get_word_vector_dict()

In [5]:
np.isnan(vectors).any()

False

In [7]:
#Gender specific vocabulary:
gender_specific=[]
with open('./Data/male_word_file.txt') as f:
    gender_specific = [line.strip() for line in f]

with open('./Data/female_word_file.txt') as f:
    for l in f:
        gender_specific.append(l.strip())

with codecs.open('./Data/gender_specific_full.json') as f:
    gender_specific.extend(json.load(f))

In [1]:
# Getting a limited vocabulary to debias the embeddings.
vocab_cleaned, vectors_cleaned, word2idx_cleaned, dict_vec_cleaned = glove.limit_vocab(vectors, word2idx, vocab, exclude=gender_specific)

NameError: name 'glove' is not defined

In [8]:
#Getting the definitional sets to calculate afterwards the gender direction. The first 10 gender sets were proposed by Bolukbasi et al. (2016)
#Definitional sets for race where proposed by Manzini et al. in Multiclass debiasing of embeddings: https://github.com/TManzini/DebiasMulticlassWordEmbedding/blob/master/Debiasing/data/vocab/race_attributes_optm.json

def_sets={
    "def_sets_gender" : [
    ['she', 'he'], ['herself', 'himself'], ['her', 'his'], ['daughter', 'son'], ['girl', 'boy'],
    ['mother', 'father'], ['woman', 'man'], ['mary', 'john'], ['gal', 'guy'], ['female', 'male'],['aunt', 'uncle']],
    
    "def_sets_race":[
		["black", "caucasian", "asian"],
		["african", "caucasian", "asian"],
		["black", "white", "asian"],
		["africa", "america", "asia"],
		["africa", "america", "china"],
		["africa", "europe", "asia"]
    ], 
    "def_sets_religion":[
		["judaism", "christianity", "islam"],
		["jew", "christian", "muslim"],
    ["synagogue", "church", "mosque"],
    ["torah", "bible", "quran"],
    ["rabbi", "priest", "imam"]
	]}

In [9]:
#Equalizing pairs were first published by Bolukbasi et al. in https://github.com/tolga-b/debiaswe/blob/master/data/equalize_pairs.json
equalizing_list=[["monastery", "convent"], ["spokesman", "spokeswoman"], ["Catholic_priest", "nun"], ["Dad", "Mom"], ["Men", "Women"], ["councilman", "councilwoman"], ["grandpa", "grandma"], ["grandsons", "granddaughters"], ["prostate_cancer", "ovarian_cancer"], ["testosterone", "estrogen"], ["uncle", "aunt"], ["wives", "husbands"], ["Father", "Mother"], ["Grandpa", "Grandma"], ["He", "She"], ["boy", "girl"], ["boys", "girls"], ["brother", "sister"], ["brothers", "sisters"], ["businessman", "businesswoman"], ["chairman", "chairwoman"], ["colt", "filly"], ["congressman", "congresswoman"], ["dad", "mom"], ["dads", "moms"], ["dudes", "gals"], ["ex_girlfriend", "ex_boyfriend"], ["father", "mother"], ["fatherhood", "motherhood"], ["fathers", "mothers"], ["fella", "granny"], ["fraternity", "sorority"], ["gelding", "mare"], ["gentleman", "lady"], ["gentlemen", "ladies"], ["grandfather", "grandmother"], ["grandson", "granddaughter"], ["he", "she"], ["himself", "herself"], ["his", "her"], ["king", "queen"], ["kings", "queens"], ["male", "female"], ["males", "females"], ["man", "woman"], ["men", "women"], ["nephew", "niece"], ["prince", "princess"], ["schoolboy", "schoolgirl"], ["son", "daughter"], ["sons", "daughters"], ["twin_brother", "twin_sister"]]

### Hard-Debias Algorithm

#### Gender

In [11]:
# To compute the gender bias, we need to get the embeddings of "he" and "she"
he_embed = dict_vectors['he']
she_embed = dict_vectors['she']

# Using the gender bias function to compute the bias of all the words in the limited dataset
#We create a dictionary with the word as key and the bias as value
gender_bias_original = utils.compute_bias(dict_vec_cleaned, he_embed, she_embed)

NameError: name 'dict_vec_cleaned' is not defined

In [16]:
def prepare_def_sets_subspace(list_def_sets):
  def_sets={i: v for i, v in enumerate(list_def_sets)}
  return def_sets

In [19]:
def_set_gender=utils.prepare_def_sets_subspace(def_sets["def_sets_gender"])

In [39]:
import HardDebias

In [40]:
debiased_vectors= HardDebias.hard_debias(vectors,
                             dict_vec_cleaned, 
                             word2idx_cleaned,
                             vocab_cleaned, 
                             equalizing_list, 
                             def_set_gender,
                             1
                             )

[[ 0.418     0.24968  -0.41242  ... -0.18411  -0.11514  -0.78581 ]
 [ 0.013441  0.23682  -0.16899  ... -0.56657   0.044691  0.30392 ]
 [ 0.15164   0.30177  -0.16763  ... -0.35652   0.016413  0.10216 ]
 ...
 [-0.51181   0.058706  1.0913   ... -0.25003  -1.125     1.5863  ]
 [-0.75898  -0.47426   0.4737   ...  0.78954  -0.014116  0.6448  ]
 [ 0.072617 -0.51393   0.4728   ... -0.18907  -0.59021   0.55559 ]]
number of words considered: 2
Running PCA with 1 components


ValueError: shapes (50,) and (1,50) not aligned: 50 (dim 0) != 1 (dim 0)

In [ ]:
debiased_vocab_limited, debiased_vectors_limited, debiased_word2idx_limited = glove.limit_vocab(debiased_vectors, word2idx, vocab, exclude = gender_specific)

###  Double-Hard Debias

In [13]:
biased_w2i, biased_vocab, female_words, male_words, y_true=utils.getting_biased_words(gender_bias_original, def_sets["def_sets_gender"], 1000,word2idx_cleaned)

np.isnan(vectors).any()

In [ ]:
precisions, optimal_direction = getting_optimal_direction(vectors, word2idx, biased_w2i, biased_vocab, male_words, female_words, y_true)

In [ ]:
dh_debiased_vectors= DoubleHardDebias.hard_debias(vectors, word2idx, word2idx, vocab, [optimal_direction], gendered_pairs)

In [ ]:
dh_debiased_vocab_limited, dh_debiased_vectors_limited, dh_debiased_word2idx_limited = glove.limit_vocab(dh_debiased_vectors, word2idx, vocab, exclude = gender_specific)

### INLP

num_vectors_per_class = 7500

#gender_direction=algorithm1.find_gender_direction(vectors, word2idx_cleaned)

gender_vecs = [glove.model[p[0]] - glove.model[p[1]] for p in gendered_pairs]
pca = PCA(n_components=1)
pca.fit(gender_vecs)
gender_direction = pca.components_[0]

gender_unit_vec = gender_direction/np.linalg.norm(gender_direction)
masc_words_and_scores, fem_words_and_scores, neut_words_and_scores = getting_classes_for_INLP(gender_vector=gender_direction, model=glove.model, n = 7500)

masc_words, masc_scores = list(zip(*masc_words_and_scores))
neut_words, neut_scores = list(zip(*neut_words_and_scores))
fem_words, fem_scores = list(zip(*fem_words_and_scores))
masc_vecs, fem_vecs = glove.get_vectors_from_list(masc_words), glove.get_vectors_from_list(fem_words)
neut_vecs = glove.get_vectors_from_list(neut_words)

n = min(3000, num_vectors_per_class)
all_significantly_biased_words = masc_words[:n] + fem_words[:n]
all_significantly_biased_vecs =  np.concatenate((masc_vecs[:n], fem_vecs[:n]))
all_significantly_biased_labels = np.concatenate((np.ones(n, dtype = int),
                                                  np.zeros(n, dtype = int)))

all_significantly_biased_words, all_significantly_biased_vecs, all_significantly_biased_labels = sklearn.utils.shuffle(
all_significantly_biased_words, all_significantly_biased_vecs, all_significantly_biased_labels)
#print(np.random.choice(masc_words, size = 75))
print("TOP MASC:",masc_words[:50] )
print("-------------------------")
print("TOP FEM:", fem_words[:50])
print("-------------------------")
print("Random Neutral:", neut_words[:50])

In [ ]:
random.seed(0)
np.random.seed(0)

X = np.concatenate((masc_vecs, fem_vecs, neut_vecs), axis = 0)
#X = (X - np.mean(X, axis = 0, keepdims = True)) / np.std(X, axis = 0)
y_masc = np.ones(masc_vecs.shape[0], dtype = int)
y_fem = np.zeros(fem_vecs.shape[0], dtype = int)
y_neut = -np.ones(neut_vecs.shape[0], dtype = int)
#y = np.concatenate((masc_scores, fem_scores, neut_scores))#np.concatenate((y_masc, y_fem))
y = np.concatenate((y_masc, y_fem, y_neut))
X_train_dev, X_test, y_train_dev, Y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train, X_dev, Y_train, Y_dev = sklearn.model_selection.train_test_split(X_train_dev, y_train_dev, test_size = 0.3, random_state = 0)
print("Train size: {}; Dev size: {}; Test size: {}".format(X_train.shape[0], X_dev.shape[0], X_test.shape[0]))

In [ ]:
gender_clf = LinearSVC
#gender_clf = SGDClassifier
#gender_clf = LogisticRegression
#gender_clf = LinearDiscriminantAnalysis
#gender_clf = Perceptron

params_svc = {'penalty': 'l2','fit_intercept': False, 'class_weight': None, "dual": False, 'random_state': 0}
params_sgd = {'penalty': 'l2','fit_intercept': False, 'class_weight': None, 'max_iter': 1000, 'random_state': 0}
params = params_svc
#params = {'loss': 'hinge', 'n_jobs': 16, 'penalty': 'l2', 'max_iter': 2500, 'random_state': 0}
#params = {}
n = 35
min_acc = 0
dropout_rate = 0


In [ ]:
P, rowspace_projs, Ws = INLP.get_debiasing_projection(gender_clf, params, n, 50, min_acc,
                                    X_train, Y_train, X_dev, Y_dev, Y_train_main=None, Y_dev_main=None, 
                                       dropout_rate = 0)